In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Generate the instance of SBM
n = [5, 5, 5]
groups = [[range(sum(n[:i]), sum(n[:i+1]))] for i in range(0, len(n))] #range(0, 5), range(5, 10), range(10, 15)

P = [[0.5, 0.3, 0.1],
     [0.3, 0.7, 0.1],
     [0.1, 0.1, 0.3]]

G = nx.stochastic_block_model(n, P)

#means of Gaussian distribution for group-group weights of edges:
W = [[0.9, 0.5, 0.2],
    [0.5, 0.8, 0.1],
    [0.2, 0.1, 0.9]]

In [95]:
 #range(0, 5), range(5, 10), range(10, 15)
groups

[[range(0, 5)], [range(5, 10)], [range(10, 15)]]

In [3]:
list(G.adj[0])

[2, 9, 13]

In [67]:
def group_belonging(i: int, n: list):
    if i >= sum(n):
        raise ValueError("Such item does not belong in n")
        
    for j in range(1, len(n)):
        if sum(n[:j]) > i:
            return j-1
    return len(n)-1

In [69]:
group_belonging(13, n)

2

In [70]:
def generate_weights_matrix(G: nx.Graph, W: np.array, n: list) -> np.array:
    ''''''
    
    #cumulative_n = [sum(n[:i]) for i in range(len(n)+1)]
    
    weights_matrix = np.full((sum(n), sum(n)), -1.0)

    #For each node:
    for key in range(sum(n)):
        group1 = group_belonging(key, n)

        #For each other node:
        for potential_neighbour in range(sum(n)):

            #If the node is a neighbour:
            if (potential_neighbour in G.adj[key]):

                #If this is the first time we see the edge - generate weight:
                if potential_neighbour > key:
                    group2 = group_belonging(potential_neighbour, n)
                    mean = W[group1][group2] #Retrieve the mean of the distribution from the weight matrix
                    weight = round(abs(np.random.normal(loc=mean, scale=1.0)), 2) #Using the absolute value here
                    weights_matrix[key][potential_neighbour] = weight

                #Else, copy the weight given before:
                else:
                    weights_matrix[key][potential_neighbour] = weights_matrix[potential_neighbour][key]

            elif potential_neighbour == key: #Self-loops' weight goes here
                weights_matrix[key][potential_neighbour] = 0

            else: #Regular disconnected nodes
                weights_matrix[key][potential_neighbour] = 0
                
    return weights_matrix

In [71]:
weights_matrix

array([[0.  , 0.  , 1.8 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 2.37, 0.  ,
        0.  , 0.  , 0.15, 0.  ],
       [0.  , 0.  , 0.  , 1.5 , 1.68, 1.28, 0.  , 1.78, 0.38, 0.3 , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [1.8 , 0.  , 0.  , 0.  , 0.09, 0.  , 0.  , 1.02, 0.  , 0.  , 0.  ,
        0.  , 0.54, 0.  , 0.  ],
       [0.  , 1.5 , 0.  , 0.  , 2.55, 0.51, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.73, 0.  ],
       [0.  , 1.68, 0.09, 2.55, 0.  , 0.  , 0.  , 0.  , 0.87, 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.28, 0.  , 0.51, 0.  , 0.  , 4.88, 0.  , 0.21, 1.31, 0.04,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 4.88, 0.  , 1.4 , 2.49, 0.01, 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.78, 1.02, 0.  , 0.  , 0.  , 1.4 , 0.  , 0.  , 0.48, 0.35,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.38, 0.  , 0.  , 0.87, 0.21, 2.49, 0.  , 0.  , 0.04, 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [2.37, 0.3 , 0.  , 0.  , 0.  ,

In [73]:
#G.add_weighted_edges_from(weights_matrix)

In [4]:
#Wrong way, only works with the adjacency matrix (?)
A = nx.to_numpy_array(G)
A

In [81]:
eigenvalues, eigenvectors = np.linalg.eig(weights_matrix)

In [82]:
eigenvalues

array([ 6.46083627, -6.0223001 ,  3.67884888,  3.05867111, -3.06704262,
       -2.83422951, -2.18446496,  1.17316209,  0.9809398 ,  0.72879961,
        0.38339434, -1.09218456, -0.04462008, -0.78722524, -0.43258504])

In [83]:
eigenvectors

array([[ 8.34746870e-02,  8.95474870e-02,  1.48017006e-02,
         6.57921943e-01,  6.18797839e-01, -2.42940402e-01,
         1.38798126e-01,  1.94900950e-01,  9.06524948e-02,
         1.07993063e-01, -1.47919440e-01,  7.81385319e-02,
        -6.46955640e-02,  1.73580826e-02,  1.99438913e-02],
       [ 2.94795815e-01, -2.11738169e-01, -4.24909233e-01,
         1.44147703e-02, -3.49119136e-02, -1.03955473e-01,
         6.77861171e-01, -1.54903513e-01, -2.48417401e-01,
        -5.93509912e-02,  2.38001461e-01, -1.03869780e-01,
         1.27194585e-01, -2.94913432e-03,  2.07691487e-01],
       [ 6.30495798e-02, -6.84518161e-02, -2.93130723e-02,
         4.54883242e-01, -4.34831157e-01,  1.43288172e-01,
         1.23405415e-01, -3.07960117e-01, -1.60558694e-02,
        -1.18958747e-01, -4.29928555e-01,  2.39567368e-01,
         1.28963695e-01, -1.59014059e-01, -4.06602936e-01],
       [ 1.91199274e-01, -9.44117984e-03, -5.51793331e-01,
        -6.94318923e-02, -1.71724518e-01, -6.02755924